In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

from Project.Database import Db

In [54]:
year1_hour, meta1_hour  = Db.load_data(year=1, hourly=True, meta=True)
year2_hour, meta2_hour = Db.load_data(year=2, hourly=True, meta=True)

In [55]:
production = []
consumption = []

for df in [year1_hour, year2_hour]:
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(unit="h", arg=(df["Timestamp"].str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[1]).astype(int))
    df["Season"] = "summer"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([3,4,5])), "Season"] = "spring"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([9,10,11])), "Season"] = "fall"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([1,2,12])), "Season"] = "winter"
    production.append(meta1_hour.loc[(meta1_hour['Parameter'] == "Power_Electrical") & (
    meta1_hour["Description"].str.contains("produced"))]["Unnamed: 0"].tolist())

In [37]:
production_year1 = meta1_hour.loc[(meta1_hour['Parameter'] == "Power_Electrical") & (
    meta1_hour["Description"].str.contains("produced"))]["Unnamed: 0"].tolist()

consumption_year1 = meta1_hour.loc[((meta1_hour['Parameter'] == "Power_Electrical") | (meta1_hour['Parameter'] == "Power_Thermal")) & (
    meta1_hour["Description"].str.contains("power consumption" or "used"))]["Unnamed: 0"].tolist()

In [38]:
production_year2 = meta2_hour.loc[(meta2_hour['Parameter'] == "Power_Electrical") & (
    meta2_hour["Description"].str.contains("produced"))]["Unnamed: 0"].tolist()

consumption_year2 = meta2_hour.loc[((meta2_hour['Parameter'] == "Power_Electrical") | (meta2_hour['Parameter'] == "Power_Thermal")) & (
    meta2_hour["Description"].str.contains("power consumption" or "used"))]["Unnamed: 0"].tolist()

In [39]:
year1_hour["Production"] = year1_hour[production_year1].sum(1)
year1_hour["Consumption"] = year1_hour[consumption_year1].sum(1)

In [40]:
year2_hour["Production"] = year2_hour[production_year2].sum(1)
year2_hour["Consumption"] = year2_hour[consumption_year2].sum(1)

In [41]:
year1_hour["Season_order"] = [0 if Season == "spring" else 1 if Season == "summer" else 2 if Season == "fall" else 3 for Season in year1_hour["Season"]]
year1_hour.sort_values(by="Season_order", ascending=True, inplace=True)

In [43]:
year2_hour["Season_order"] = [0 if Season == "spring" else 1 if Season == "summer" else 2 if Season == "fall" else 3 for Season in year2_hour["Season"]]
year2_hour.sort_values(by="Season_order", ascending=True, inplace=True)

In [44]:
# Seasonal violin plots of production for both year 1 and 2
fig = px.violin(year1_hour, x="Season", y="Production", color="Season")
fig.write_html("Season_production_violin_year1.html")

fig = px.violin(year2_hour, x="Season", y="Production", color="Season")
fig.write_html("Season_production_violin_year2.html")

In [45]:
# Seasonal violin plots of consumption for both year 1 and 2
fig = px.violin(year1_hour, x="Season", y="Consumption", color="Season")
fig.write_html("Season_consumption_violin_year1.html")

fig = px.violin(year2_hour, x="Season", y="Consumption", color="Season")
fig.write_html("Season_consumption_violin_year2.html")